<a href="https://colab.research.google.com/github/haddybhaiya/sem-i-con/blob/main/train_semicon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install torch torchvision timm albumentations opencv-python


In [ ]:
import torch
import timm
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader , random_split
import torch.nn as nn
import torch.optim as optim

DATASET_PATH = "/content/drive/MyDrive/synthetic_dataset"
MODEL_SAVE_PATH = "/content/drive/MyDrive/edge_model.pth"

CLASSES = ["clean","bridge","cmp","crack","open","ler","via","other"]
NUM_CLASSES = 8

IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 20
WARMUP_EPOCHS = 5   # backbone frozen
LR = 1e-4

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", DEVICE)


Using device: cuda


In [ ]:
train_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

val_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])


In [ ]:
full_dataset = datasets.ImageFolder(DATASET_PATH, transform=train_transform)

train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size

train_dataset, val_dataset = random_split(
    full_dataset, [train_size, val_size]
)

# override transform for validation
val_dataset.dataset.transform = val_transform

train_loader = DataLoader(
    train_dataset, batch_size=BATCH_SIZE,
    shuffle=True, num_workers=2
)

val_loader = DataLoader(
    val_dataset, batch_size=BATCH_SIZE,
    shuffle=False, num_workers=2
)

print("Train samples:", len(train_dataset))
print("Val samples:", len(val_dataset))


Train samples: 1920
Val samples: 480


In [ ]:
model = timm.create_model(
    "mobilenetv3_small_100",
    pretrained=True,
    num_classes=NUM_CLASSES
)

model = model.to(DEVICE)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/10.2M [00:00<?, ?B/s]

In [ ]:
for name, param in model.named_parameters():
    if "classifier" not in name:
        param.requires_grad = False

print("Backbone frozen. Training classifier head only.")


Backbone frozen. Training classifier head only.


In [ ]:
criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(
    filter(lambda p: p.requires_grad, model.parameters()),
    lr=LR
)


In [ ]:
def train_epoch(loader):
    model.train()
    total_loss, correct, total = 0, 0, 0

    for x, y in loader:
        x, y = x.to(DEVICE), y.to(DEVICE)

        optimizer.zero_grad()
        out = model(x)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        preds = out.argmax(1)
        correct += (preds == y).sum().item()
        total += y.size(0)

    return total_loss / len(loader), correct / total


def eval_epoch(loader):
    model.eval()
    total_loss, correct, total = 0, 0, 0

    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(DEVICE), y.to(DEVICE)
            out = model(x)
            loss = criterion(out, y)

            total_loss += loss.item()
            preds = out.argmax(1)
            correct += (preds == y).sum().item()
            total += y.size(0)

    return total_loss / len(loader), correct / total


In [ ]:
for epoch in range(EPOCHS):

    # unfreeze backbone after warm-up
    if epoch == WARMUP_EPOCHS:
        print("\nUnfreezing backbone for fine-tuning...\n")
        for param in model.parameters():
            param.requires_grad = True

        optimizer = optim.Adam(model.parameters(), lr=LR * 0.1)

    train_loss, train_acc = train_epoch(train_loader)
    val_loss, val_acc = eval_epoch(val_loader)

    print(
        f"Epoch {epoch+1}/{EPOCHS} | "
        f"Train Acc: {train_acc:.4f} | "
        f"Val Acc: {val_acc:.4f}"
    )


Epoch 1/20 | Train Acc: 0.1141 | Val Acc: 0.1729
Epoch 2/20 | Train Acc: 0.1635 | Val Acc: 0.2042
Epoch 3/20 | Train Acc: 0.2104 | Val Acc: 0.2375
Epoch 4/20 | Train Acc: 0.2812 | Val Acc: 0.3167
Epoch 5/20 | Train Acc: 0.3531 | Val Acc: 0.4021

Unfreezing backbone for fine-tuning...

Epoch 6/20 | Train Acc: 0.6891 | Val Acc: 0.8458
Epoch 7/20 | Train Acc: 0.8917 | Val Acc: 0.9354
Epoch 8/20 | Train Acc: 0.9385 | Val Acc: 0.9646
Epoch 9/20 | Train Acc: 0.9542 | Val Acc: 0.9583
Epoch 10/20 | Train Acc: 0.9646 | Val Acc: 0.9604
Epoch 11/20 | Train Acc: 0.9781 | Val Acc: 0.9750
Epoch 12/20 | Train Acc: 0.9854 | Val Acc: 0.9812
Epoch 13/20 | Train Acc: 0.9844 | Val Acc: 0.9750
Epoch 14/20 | Train Acc: 0.9854 | Val Acc: 0.9854
Epoch 15/20 | Train Acc: 0.9906 | Val Acc: 0.9875
Epoch 16/20 | Train Acc: 0.9917 | Val Acc: 0.9896
Epoch 17/20 | Train Acc: 0.9901 | Val Acc: 0.9771
Epoch 18/20 | Train Acc: 0.9958 | Val Acc: 0.9771
Epoch 19/20 | Train Acc: 0.9969 | Val Acc: 0.9771
Epoch 20/20 | Trai

In [ ]:
torch.save(model.state_dict(), MODEL_SAVE_PATH)
print("Model saved to:", MODEL_SAVE_PATH)


Model saved to: /content/drive/MyDrive/edge_model.pth
